In [3]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 5.4MB/s 


In [1]:
!pip install datashader

     |████████████████████████████████| 15.5MB 214kB/s 
     |████████████████████████████████| 1.6MB 49.0MB/s 
     |████████████████████████████████| 81kB 12.0MB/s 
     |████████████████████████████████| 92kB 12.0MB/s 
     |████████████████████████████████| 665kB 47.2MB/s 
     |████████████████████████████████| 102kB 13.3MB/s 
  Created wheel for datashape: filename=datashape-0.5.2-cp36-none-any.whl size=59430 sha256=90611cd9ccccc4428dc0a388a27d6c3f2d27f9637ef66fca9ee08a0387dbcc09
  Stored in directory: /root/.cache/pip/wheels/8d/06/05/c1cba3d57bdcfd3960e3f60a9fdc97e4baef2ef09af0ad1ef8
  Created wheel for locket: filename=locket-0.2.0-cp36-none-any.whl size=4040 sha256=7f118e063b76ec1c6138a690f118945454aefd36f3d10bf9a399ed29efc2fc8d
  Stored in directory: /root/.cache/pip/wheels/26/1e/e8/4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=11ad7bf3ad6d7d03e5f743764d9fe6562e54d0b379a876a70a22

In [4]:
import nltk
from collections import Counter
from nltk import bigrams
from nltk import trigrams
import os
import pandas as pd

import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt

import nltk
import numpy as np
import re
import pandas as pd 
import pylab as pl
import matplotlib.pyplot as plt

from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD

from sklearn import metrics
from mpl_toolkits.mplot3d import Axes3D

from textblob import TextBlob

from matplotlib import pyplot

#plt.style.use('fivethirtyeight')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'


from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

import datashader as ds
import datashader.transfer_functions as tf

In [5]:
df = pd.read_csv("/content/qanda_202010222231.csv")
df.head()

,id,company_id,calendar_id,number,executive_id,analyst_id,question,answer,exclude
0,146369,18,163,1,73.0,22,"Thank you, good morning.",Good morning. Good morning.,False
1,146864,23,176,1,293.0,472,"Hi, good morning, guys. I wanted to ask about ...","Hey, John. This is Clark. I'll take that. Kell...",False
2,141372,5,23,1,15.0,62,Group,Good morning Joe. Good morning Joe.,False
3,147314,39,188,1,175.0,21,Good morning. Just looking to get a little bit...,"Yeah, John how are you doing. It's John here. ...",False
4,144010,33,95,1,218.0,3,Good afternoon. Just wanted to start on the br...,That is correct. Yes. We expect savings o...,False


In [6]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [7]:
df['Clean_text'] = np.vectorize(remove_pattern)(df['question'], "@[\w]*")

In [61]:
df['Clean_text'] = np.vectorize(remove_pattern)(df['question'], "morning")

In [62]:
df['Clean_text'].head()

0                                    Thank you, good .
1    Hi, good , guys. I wanted to ask about the loa...
2                                                Group
3    Good . Just looking to get a little bit more c...
4    Good afternoon. Just wanted to start on the br...
Name: Clean_text, dtype: object

In [9]:
df['Clean_text'] = df['Clean_text'].str.replace("[^a-zA-Z#]", " ")
df['Clean_text'] = df['Clean_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

In [10]:
tokenized_test = df['Clean_text'].apply(lambda x: x.split())

In [11]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

tokenized_test = tokenized_test.apply(lambda x: [i for i in x]) # stemming
tokenized_test.head()

0                               [Thank, good, morning]
1    [good, morning, guys, wanted, about, loan, gro...
2                                              [Group]
3    [Good, morning, Just, looking, little, more, c...
4    [Good, afternoon, Just, wanted, start, branch,...
Name: Clean_text, dtype: object

In [12]:
for i in range(len(tokenized_test)):
    tokenized_test[i] = ' '.join(tokenized_test[i])

df['Clean_text'] = tokenized_test

In [14]:
df.loc[:,('question','Clean_text')].head()

,question,Clean_text
0,"Thank you, good morning.",Thank good morning
1,"Hi, good morning, guys. I wanted to ask about ...",good morning guys wanted about loan growth sho...
2,Group,Group
3,Good morning. Just looking to get a little bit...,Good morning Just looking little more color de...
4,Good afternoon. Just wanted to start on the br...,Good afternoon Just wanted start branch closur...


In [15]:
tok = []
for i in df['Clean_text']:
    for j in i.split():
        tok.append(j.replace("'","").replace('"',""))
print(tok[0:100])

['Thank', 'good', 'morning', 'good', 'morning', 'guys', 'wanted', 'about', 'loan', 'growth', 'showed', 'good', 'loan', 'growth', 'this', 'quarter', 'wondering', 'could', 'break', 'down', 'little', 'more', 'between', 'commercial', 'side', 'consumer', 'side', 'Specifically', 'commercial', 'some', 'your', 'peers', 'point', 'elevated', 'paydowns', 'notice', 'pure', 'basis', 'your', 'balances', 'were', 'down', 'just', 'wondering', 'what', 'seeing', 'there', 'then', 'mortgage', 'side', 'seemed', 'like', 'resi', 'mortgage', 'driver', 'loan', 'growth', 'this', 'quarter', 'What', 'were', 'some', 'factors', 'there', 'Group', 'Good', 'morning', 'Just', 'looking', 'little', 'more', 'color', 'deposit', 'efforts', 'this', 'quarter', 'what', 'areas', 'what', 'types', 'deposits', 'pushing', 'there', 'broad', 'based', 'increase', 'state', 'rate', 'then', 'what', 'your', 'outlook', 'there', 'there', 'continued', 'push', 'beyond', 'your', 'existing', 'national']


In [16]:
Counter(tok).most_common()

[('that', 13074),
 ('just', 6473),
 ('about', 4055),
 ('your', 3983),
 ('what', 3445),
 ('Okay', 3440),
 ('then', 3318),
 ('quarter', 3089),
 ('there', 2963),
 ('think', 2906),
 ('this', 2897),
 ('guys', 2332),
 ('some', 2322),
 ('have', 2318),
 ('kind', 2281),
 ('Thanks', 2240),
 ('year', 2114),
 ('growth', 2070),
 ('morning', 2067),
 ('more', 1926),
 ('from', 1896),
 ('like', 1854),
 ('with', 1802),
 ('question', 1749),
 ('going', 1587),
 ('maybe', 1567),
 ('could', 1556),
 ('little', 1540),
 ('good', 1520),
 ('Thank', 1516),
 ('loan', 1495),
 ('Good', 1460),
 ('guess', 1400),
 ('much', 1349),
 ('would', 1325),
 ('right', 1269),
 ('know', 1228),
 ('million', 1220),
 ('terms', 1200),
 ('where', 1149),
 ('rate', 1137),
 ('Just', 1124),
 ('back', 1112),
 ('deposit', 1061),
 ('those', 1029),
 ('follow', 1026),
 ('last', 972),
 ('over', 969),
 ('look', 958),
 ('That', 956),
 ('very', 893),
 ('helpful', 882),
 ('down', 880),
 ('color', 880),
 ('into', 865),
 ('been', 863),
 ('should', 862)

In [17]:
def calculate_sentiment(Clean_text):
    return TextBlob(Clean_text).sentiment

In [18]:
def calculate_sentiment_analyser(Clean_text):    
    return analyser.polarity_scores(Clean_text)

In [19]:
df['sentiment']=df.Clean_text.apply(calculate_sentiment)

In [20]:
df['sentiment_analyser']=df.Clean_text.apply(calculate_sentiment_analyser)

In [21]:
df.loc[:,('Clean_text','sentiment','sentiment_analyser')].head(20)

,Clean_text,sentiment,sentiment_analyser
0,Thank good morning,"(0.7, 0.6000000000000001)","{'neg': 0.0, 'neu': 0.156, 'pos': 0.844, 'comp..."
1,good morning guys wanted about loan growth sho...,"(0.17951940035273364, 0.36419753086419754)","{'neg': 0.0, 'neu': 0.774, 'pos': 0.226, 'comp..."
2,Group,"(0.0, 0.0)","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3,Good morning Just looking little more color de...,"(0.25416666666666665, 0.43541666666666673)","{'neg': 0.0, 'neu': 0.841, 'pos': 0.159, 'comp..."
4,Good afternoon Just wanted start branch closur...,"(0.32857142857142857, 0.37857142857142856)","{'neg': 0.0, 'neu': 0.756, 'pos': 0.244, 'comp..."
5,good morning Paul hoping just clarify outlook ...,"(0.14629629629629629, 0.2592592592592593)","{'neg': 0.05, 'neu': 0.563, 'pos': 0.387, 'com..."
6,Yeah thanks Good morning everyone Good hear th...,"(0.575, 0.5)","{'neg': 0.133, 'neu': 0.292, 'pos': 0.575, 'co..."
7,Good morning Jamie wondering could give some i...,"(0.12499999999999999, 0.275)","{'neg': 0.0, 'neu': 0.855, 'pos': 0.145, 'comp..."
8,good morning everybody,"(0.7, 0.6000000000000001)","{'neg': 0.0, 'neu': 0.408, 'pos': 0.592, 'comp..."
9,Good morning guess just follow your comments a...,"(-0.08000000000000003, 0.6533333333333334)","{'neg': 0.062, 'neu': 0.751, 'pos': 0.186, 'co..."


In [52]:
#df.to_csv("/content/question_with_sentiment.csv")

In [22]:
s = pd.DataFrame(index = range(0,len(df)),columns= ['compound_score','compound_score_sentiment'])

In [23]:
for i in range(0,len(df)): 
  s['compound_score'][i] = df['sentiment_analyser'][i]['compound']
  
  if (df['sentiment_analyser'][i]['compound'] < 0.2 ):
    s['compound_score_sentiment'][i] = 'Negative'    
  if (df['sentiment_analyser'][i]['compound'] >= 0.5):
    s['compound_score_sentiment'][i] = 'Positive'
  if ((df['sentiment_analyser'][i]['compound'] >= 0.2) & (df['sentiment_analyser'][i]['compound'] < 0.5)):
    s['compound_score_sentiment'][i] = 'Neutral'

In [24]:
df['compound_score'] = s['compound_score']

In [25]:
df['compound_score_sentiment'] = s['compound_score_sentiment']

In [26]:
df.sample(n=20).head(20)

,id,company_id,calendar_id,number,executive_id,analyst_id,question,answer,exclude,Clean_text,sentiment,sentiment_analyser,compound_score,compound_score_sentiment
5410,144415,9,108,16,31.0,195,Got it.,Now embedded obviously if there is a fixed com...,False,,"(0.0, 0.0)","{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...",0,Negative
3851,144635,27,116,11,222.0,189,"Okay. And then one more impossible question, J...","So, Glenn, I'm glad you acknowledged that it's...",False,Okay then more impossible question maybe Could...,"(0.09155092592592594, 0.4428240740740741)","{'neg': 0.072, 'neu': 0.697, 'pos': 0.231, 'co...",0.8952,Positive
11927,143775,41,88,44,204.0,23,"Okay. And then turning to credit, I hear you o...",Pete? We do look our qualitative reserves...,False,Okay then turning credit hear credit cost been...,"(0.45, 0.6333333333333333)","{'neg': 0.056, 'neu': 0.613, 'pos': 0.331, 'co...",0.8807,Positive
10692,148216,3,215,35,36.0,98,"Okay, great, that's helpful background. And ag...","There is some marginal deterioration, but we'r...",False,Okay great that helpful background again with ...,"(0.4625, 0.5625)","{'neg': 0.0, 'neu': 0.652, 'pos': 0.348, 'comp...",0.8807,Positive
5115,146354,22,162,15,87.0,300,Just to talk a little bit about the loan growt...,"Sure. This is Chris. First and foremost, some ...",False,Just talk little about loan growth loan dynami...,"(0.18214285714285713, 0.5952380952380951)","{'neg': 0.0, 'neu': 0.762, 'pos': 0.238, 'comp...",0.9417,Positive
8647,148532,33,226,26,219.0,32,"Yeah. That's helpful. Okay, great, thanks for ...",we do expect to use a quarters worth of credit...,False,Yeah That helpful Okay great thanks that then ...,"(0.25, 0.31190476190476196)","{'neg': 0.0, 'neu': 0.571, 'pos': 0.429, 'comp...",0.9382,Positive
11273,150227,2,273,39,7.0,14,Got it.,"So again, not a real significant impact to us ...",False,,"(0.0, 0.0)","{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...",0,Negative
5850,144718,20,118,17,85.0,156,Thanks for the call guys.,I try.,False,Thanks call guys,"(0.2, 0.2)","{'neg': 0.0, 'neu': 0.408, 'pos': 0.592, 'comp...",0.4404,Neutral
1406,143360,40,78,5,183.0,146,Okay. So at normal response to a cut might be ...,"Yeah, somewhere around there.",False,Okay normal response might less basis points b...,"(0.16111111111111112, 0.4055555555555555)","{'neg': 0.0, 'neu': 0.853, 'pos': 0.147, 'comp...",0.2263,Neutral
10331,146471,17,165,33,91.0,456,Any update on potential impact for what you mi...,Yeah. We'll have that in the second quarter. W...,False,update potential impact what might have your a...,"(0.0, 1.0)","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0,Negative


In [58]:
#df.to_csv("/content/question_with_sentiment_v2.csv")

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [28]:
# Considering 3 grams and mimnimum frq as 0
tf_idf_vect = CountVectorizer(analyzer='word',ngram_range=(1,2),stop_words='english', min_df = 0.0001)

In [29]:
tf_idf_vect.fit(df['Clean_text'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=0.0001,
                ngram_range=(1, 2), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [30]:
desc_matrix = tf_idf_vect.transform(df["Clean_text"])

In [31]:
from sklearn.cluster import KMeans

num_clusters = 5
km = KMeans(n_clusters=num_clusters)
km.fit(desc_matrix)
clusters = km.labels_.tolist()

In [32]:
df['cluster'] = pd.Series(clusters)

In [33]:
df.head()

,id,company_id,calendar_id,number,executive_id,analyst_id,question,answer,exclude,Clean_text,sentiment,sentiment_analyser,compound_score,compound_score_sentiment,cluster
0,146369,18,163,1,73.0,22,"Thank you, good morning.",Good morning. Good morning.,False,Thank good morning,"(0.7, 0.6000000000000001)","{'neg': 0.0, 'neu': 0.156, 'pos': 0.844, 'comp...",0.6597,Positive,3
1,146864,23,176,1,293.0,472,"Hi, good morning, guys. I wanted to ask about ...","Hey, John. This is Clark. I'll take that. Kell...",False,good morning guys wanted about loan growth sho...,"(0.17951940035273364, 0.36419753086419754)","{'neg': 0.0, 'neu': 0.774, 'pos': 0.226, 'comp...",0.9337,Positive,0
2,141372,5,23,1,15.0,62,Group,Good morning Joe. Good morning Joe.,False,Group,"(0.0, 0.0)","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0,Negative,1
3,147314,39,188,1,175.0,21,Good morning. Just looking to get a little bit...,"Yeah, John how are you doing. It's John here. ...",False,Good morning Just looking little more color de...,"(0.25416666666666665, 0.43541666666666673)","{'neg': 0.0, 'neu': 0.841, 'pos': 0.159, 'comp...",0.7964,Positive,3
4,144010,33,95,1,218.0,3,Good afternoon. Just wanted to start on the br...,That is correct. Yes. We expect savings o...,False,Good afternoon Just wanted start branch closur...,"(0.32857142857142857, 0.37857142857142856)","{'neg': 0.0, 'neu': 0.756, 'pos': 0.244, 'comp...",0.8126,Positive,4


In [35]:
#df.to_csv("/content/question_with_sentiment_v3.csv")

In [36]:
tok = []
for i in df['Clean_text']:
    for j in i.split():
        tok.append(j.replace("'","").replace('"',""))
print(tok[0:10])

['Thank', 'good', 'morning', 'good', 'morning', 'guys', 'wanted', 'about', 'loan', 'growth']


In [40]:
data = []
for t in df['Clean_text']:
  data.append(t)
data[:10]

['Thank good morning',
 'good morning guys wanted about loan growth showed good loan growth this quarter wondering could break down little more between commercial side consumer side Specifically commercial some your peers point elevated paydowns notice pure basis your balances were down just wondering what seeing there then mortgage side seemed like resi mortgage driver loan growth this quarter What were some factors there',
 'Group',
 'Good morning Just looking little more color deposit efforts this quarter what areas what types deposits pushing there broad based increase state rate then what your outlook there there continued push beyond your existing national platform that going continue push product therefore deposit rates higher Thanks',
 'Good afternoon Just wanted start branch closures sounds like million million that guys targeting interest expense next year that based both cost saves here then additional initiatives that right',
 'good morning Paul hoping just clarify outlook 

In [43]:
Counter(tok).most_common()[:10]

[('that', 13074),
 ('just', 6473),
 ('about', 4055),
 ('your', 3983),
 ('what', 3445),
 ('Okay', 3440),
 ('then', 3318),
 ('quarter', 3089),
 ('there', 2963),
 ('think', 2906)]

In [44]:
bi_prep = ' '.join(i for i in data)

In [67]:
import re
import nltk
from nltk.util import ngrams

bi_prep = bi_prep.lower()
bi_prep = re.sub(r'[^a-zA-Z0-9\s]', ' ', bi_prep)
tokens = [token for token in bi_prep.split(" ") if token != ""]
output = list(ngrams(tokens, 6))
Counter(output).most_common()[:20]

[(('good', 'morning', 'guys', 'thanks', 'taking', 'question'), 22),
 (('thanks', 'good', 'morning', 'guys', 'good', 'morning'), 14),
 (('good', 'morning', 'guys', 'good', 'morning', 'guys'), 13),
 (('good', 'morning', 'guys', 'thanks', 'good', 'morning'), 11),
 (('good', 'morning', 'guys', 'thanks', 'taking', 'questions'), 10),
 (('okay', 'okay', 'that', 'helpful', 'then', 'just'), 10),
 (('morning', 'guys', 'thanks', 'taking', 'question', 'just'), 9),
 (('most', 'questions', 'have', 'been', 'asked', 'answered'), 9),
 (('good', 'morning', 'thanks', 'good', 'morning', 'guys'), 8),
 (('good', 'morning', 'guys', 'good', 'morning', 'good'), 8),
 (('morning', 'guys', 'good', 'morning', 'good', 'morning'), 8),
 (('good', 'morning', 'good', 'morning', 'guys', 'good'), 8),
 (('morning', 'good', 'morning', 'guys', 'good', 'morning'), 8),
 (('could', 'just', 'give', 'little', 'more', 'color'), 8),
 (('good', 'morning', 'good', 'morning', 'good', 'morning'), 7),
 (('okay', 'perfect', 'thank', 'gu